In [2]:
import os
import json
import scipy.io
import pandas as pd
from pathlib import Path
from helper import matfile_to_dic
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F


In [60]:
from Nets import MLP
from Update import LocalUpdate
import argparse
import Options
import copy
from tools import FedAvg

In [4]:
filepath=Path('12k_DE')
dataDic=matfile_to_dic(filepath)

In [41]:
print(data)

None


In [5]:
for dicson in dataDic.values():
    del dicson['__header__']
    del dicson['__version__']
    del dicson['__globals__']
dataDic

{'B007_0.mat': {'X118_DE_time': array([-0.0027614 , -0.09632403,  0.11370459, ..., -0.07829373,
         -0.14911545,  0.02111657]),
  'X118_FE_time': array([-0.24716182,  0.14279091,  0.00328727, ...,  0.01643636,
         -0.03842   , -0.16806182]),
  'X118_BA_time': array([ 0.01553163,  0.01693994, -0.03645507, ...,  0.03178754,
         -0.03243134, -0.0668343 ]),
  'X118RPM': 1796},
 'B007_1.mat': {'X119_DE_time': array([-0.0721212 ,  0.30310395,  0.03313677, ...,  0.08820228,
         -0.19687138, -0.03898443]),
  'X119_FE_time': array([ 0.07745636,  0.04561091, -0.01109455, ..., -0.01273818,
          0.04170727,  0.02753091]),
  'X119_BA_time': array([0.07327228, 0.06932902, 0.05480332, ..., 0.05753947, 0.01569258,
         0.01488783]),
  'X119RPM': 1772},
 'B007_2.mat': {'X120_DE_time': array([-0.31447441,  0.12198878,  0.39146866, ...,  0.01900491,
          0.21067836,  0.0568523 ]),
  'X120_FE_time': array([ 0.14792727, -0.04499455,  0.10806909, ..., -0.02177818,
         

In [67]:
processed_data={0:[],1:[],2:[],3:[]}
#0:B 1:IR 2:No 3:OR
for name,data in dataDic.items():
    if 'B' in name:
        for iname in data.keys():
            if 'DE' in iname:
                processed_data[0].extend(data[iname])
    elif 'IR' in name:
        for iname in data.keys():
            if 'DE' in iname:
                processed_data[1].extend(data[iname])
    elif 'No' in name:
        for iname in data.keys():
            if 'DE' in iname:
                processed_data[2].extend(data[iname])
    elif 'OR' in name:
        for iname in data.keys():
            if 'DE' in iname:
                processed_data[3].extend(data[iname])


In [67]:
print(len(processed_data))
for i in range(4):
    print(len(processed_data[i]))

4
1947286
1948115
2182450
3418357


In [68]:
#舍弃这种方式
processed_data={0:[],1:[],2:[],3:[]}
for name,data in dataDic.items():
    if 'B' in name:
        for iname in data.keys():
            if 'DE' in iname:
                processed_data[0].append(data[iname])
    elif 'IR' in name:
        for iname in data.keys():
            if 'DE' in iname:
                processed_data[1].append(data[iname])
    elif 'No' in name:
        for iname in data.keys():
            if 'DE' in iname:
                processed_data[2].append(data[iname])
    elif 'OR' in name:
        for iname in data.keys():
            if 'DE' in iname:
                processed_data[3].append(data[iname])

In [57]:
#丢弃这一步
min=9999999
def comparemin(x1,x2):
    if x1>x2:
        return x2
    else:
        return x1
for data in processed_data.values():
    for iarray in data:
        ilen=len(iarray)
        min=comparemin(ilen,min)
print(min)
#min是所有文件的DE数据中的最小值

120801


In [ ]:
#这一步去掉不要
pprocessed_data={0:[],1:[],2:[],3:[]}
for name,data in processed_data.items():
    print(name)
    print(len(data))
    for idata in data:
        pprocessed_data[name].extend(idata[0:min])

这一步的目的是保持每个mat文件的采样点的一致。防止在后面进行数据划分的时候丢失某个文件的数据

In [ ]:
#去掉不要
for i in range(len(pprocessed_data)):
    print(len(pprocessed_data[i]))

In [69]:
splited_data=[]
label=[]
for name,data in processed_data.items():
    n=len(data)
    x=n//500
    for i in range(x):
        splited_data.append(data[i*500:(i+1)*500])
        label.append(name)

In [ ]:
splited_data=torch.tensor(splited_data,dtype=torch.float32)
label=torch.tensor(label,dtype=torch.long)

In [73]:
data_and_label=torch.utils.data.TensorDataset(splited_data,label)

#可以用TensorDataset进行数据的打包
if data_type =='iid':
    num_items=int(len(label)*0.8/clients_all)
    #留下20%的数据作为实验集
    dict_users,all_idxs={},[i for i in range(len(label))]
    test_idxs=[]
    for i in range(clients_all):
        dict_users[i]=set(np.random.choice(all_idxs,num_items,replace=False))
        #随机分配数据
        all_idxs=list(set(all_idxs)-dict_users[i])
        #更新数据
    test_idxs=all_idxs


TypeError: list indices must be integers or slices, not dict

In [34]:
def normalization(data):
    for i,j in enumerate(data):
        x=(j-0)/3
        data[i]=x
    return data

In [9]:
label=normalization(label)

NameError: name 'normalization' is not defined

In [46]:
label[10000]

tensor(0.2222, dtype=torch.float64)

In [26]:
type(splited_data[0])

torch.Tensor

In [19]:
def Normalize(data):
    min_number=min(data)
    max_number=max(data)
    max_distance=max_number-min_number
    norm_data=(data-min_number)/(max_distance)
    norm_data=(norm_data-0.5)/0.5
    return norm_data

In [25]:
for x in splited_data:
    x=Normalize(x)


In [27]:
splited_data

tensor([[-0.0028, -0.0963,  0.1137,  ..., -0.0508, -0.2651,  0.0184],
        [ 0.2115, -0.0708, -0.0968,  ...,  0.0304, -0.0500,  0.0528],
        [ 0.0643, -0.0015,  0.0491,  ..., -0.0981,  0.0773,  0.1595],
        ...,
        [-0.0435, -0.2648, -0.1015,  ...,  0.0881, -0.0678, -0.0654],
        [-0.0516, -0.1994, -0.1852,  ...,  0.0041, -0.1900, -0.0727],
        [ 0.0824, -0.1023, -0.2676,  ..., -0.0971, -0.0439, -0.0118]],
       dtype=torch.float64)

In [26]:
splited_data[0][0]

tensor(-0.0028, dtype=torch.float64)

In [28]:
data_and_label=torch.utils.data.TensorDataset(splited_data,label)

In [31]:
data_type='iid'
clients_all=100
frac=0.1 #每次训练的客户占所有客户的比例

In [32]:
if data_type =='iid':
    num_items=int(len(label)*0.8/clients_all)
    #留下20%的数据作为实验集
    dict_users,all_idxs={},[i for i in range(len(label))]
    test_idxs=[]
    for i in range(clients_all):
        dict_users[i]=set(np.random.choice(all_idxs,num_items,replace=False))
        #随机分配数据
        all_idxs=list(set(all_idxs)-dict_users[i])
        #更新数据
    test_idxs=all_idxs

注意，dict_users={i:list_i},test_idxs=list
i代表第i个客户，list_i代表给第i个客户随机分配的数据。list中存储的是数据的下表index。数据储存再splited_data中。test_idxs储存的是测试集数据的下标。

In [11]:
print(len(test_idxs)/(len(test_idxs)+clients_all*len(dict_users[0])))

0.20484465508162192


In [61]:
net_glob=MLP(dim_in=500, dim_hidden=200, dim_out=4)
net_glob.train()

MLP(
  (layer_input): Linear(in_features=500, out_features=200, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (layer_hidden): Linear(in_features=200, out_features=4, bias=True)
)

In [62]:
w_glob = net_glob.state_dict()

In [63]:
epochs=10

In [64]:
loss_train=[]


In [65]:
class option:
    def __init__(self):
        self.lr=0.01
        self.momentum=0.5
        self.local_ep=10
        self.device='cpu'
        self.verbose=False
        self.local_bs=10
args=option()

In [66]:
for iter in range(epochs):
    loss_locals=[]
    w_locals = [w_glob for i in range(clients_all)]
    m=round(frac*clients_all)
    idxs_users=np.random.choice(range(clients_all),m,replace=False)
    #选出来的是客户的编号
    for idx in idxs_users:
        local=LocalUpdate(args=args,dataset=data_and_label,idxs=dict_users[idx])
        w,loss=local.train(net=copy.deepcopy(net_glob))
        w_locals[idx]=copy.deepcopy(w)
        loss_locals.append(copy.deepcopy(loss))
    w_glob=FedAvg(w_locals)
    net_glob.load_state_dict(w_glob)
    loss_avg=sum(loss_locals)/len(loss_locals)
    print('Round {:3d}, Average loss {:.3f},训练耗时 {}s'.format(iter, loss_avg,(t3-t2)))
    loss_train.append(loss_avg)    
        
        
    

RuntimeError: expected scalar type Float but found Double